## Entrenamiento de modelos ML para detección de accesos atípicos (IAM)

En este notebook se van a entrenar y evaluar diferentes modelos de detección de anomalías sobre el dataset generado en la fase de preparación. Dado que no existen etiquetas explícitas de anomalía en el dataset LANL, se van a aplicar técnicas no supervisadas / semi-supervisadas para identificar desviaciones respecto al comportamiento habitual.

In [1]:
import pandas as pd
import numpy as np
import time

t0 = time.time()

DATA_PATH = "lanl_db_feature.csv"  # ajusta si tu fichero se llama distinto
df = pd.read_csv(DATA_PATH)

df.head()

,dt,src_user,total_events,failed_events,fail_ratio,dst_hosts,src_hosts,nbhours_events,nbhours_ratio
0,1970-01-01 00:00:00,ANONYMOUS LOGON@C1065,434,0,0.0,1,82,434,1.0
1,1970-01-01 00:00:00,U561@DOM1,17,0,0.0,3,4,17,1.0
2,1970-01-01 00:00:00,U556@DOM1,20,0,0.0,5,4,20,1.0
3,1970-01-01 00:00:00,U555@DOM1,49,0,0.0,9,10,49,1.0
4,1970-01-01 00:00:00,U553@DOM1,30,0,0.0,6,7,30,1.0


In [2]:
#ocupación en memoria del csv en MB
df.memory_usage(deep=True).sum() / (1024**2) 

np.float64(4195.788251876831)

In [3]:
#Nos indica cuantos perfiles de comportamiento hay
print("Shape:", df.shape)

Shape: (42866142, 9)


In [4]:
#Se comprueban valores nulos y se ordenan las columnas que muestran las diez peores ventanas de comportamiento.
print(df.isna().mean().sort_values(ascending=False).head(10))

dt                0.0
src_user          0.0
total_events      0.0
failed_events     0.0
fail_ratio        0.0
dst_hosts         0.0
src_hosts         0.0
nbhours_events    0.0
nbhours_ratio     0.0
dtype: float64


In [5]:
#Se generan estadisticas más descriptivas para comprobar el estado del dataset
df.describe()

,total_events,failed_events,fail_ratio,dst_hosts,src_hosts,nbhours_events,nbhours_ratio
count,4.286614e+07,4.286614e+07,4.286614e+07,4.286614e+07,4.286614e+07,4.286614e+07,4.286614e+07
mean,2.444509e+01,2.980180e-01,1.314286e-02,2.900863e+00,3.393979e+00,6.157894e+00,2.396926e-01
std,1.179786e+02,1.218741e+01,1.108429e-01,2.717707e+00,1.266600e+01,6.128290e+01,4.268958e-01
min,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,1.100000e+01,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
75%,2.500000e+01,0.000000e+00,0.000000e+00,4.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
max,1.807300e+04,4.608000e+03,1.000000e+00,2.751000e+03,2.026000e+03,1.371200e+04,1.000000e+00


## Fase 1: Selección de variables

Se van a seleccionar características numéricas que más representan el comportamiento IAM:
volumen de actividad, fallos, diversidad de hosts y actividad fuera de horario.


In [6]:
#Seleccionamos las columnas con las que vamos a trabajar en el dataset, en este caso, hay columnas que no son necesarias para entrenar a nuestras IAs
DB_COLS = [
    "total_events",
    "failed_events",
    "fail_ratio",
    "dst_hosts",
    "src_hosts",
    "nbhours_events",
    "nbhours_ratio"
]

#Comprobamos que las columnas son las que se encuentran en el documento
miss = [c for c in DB_COLS if c not in df.columns]
miss

[]

In [7]:
#última limpieza del dataset antes de empezar el entrenamiento

# Se sustituyen valores infinitos por 0
db = df[DB_COLS].replace([np.inf, -np.inf], np.nan).fillna(0)

db.head()


,total_events,failed_events,fail_ratio,dst_hosts,src_hosts,nbhours_events,nbhours_ratio
0,434,0,0.0,1,82,434,1.0
1,17,0,0.0,3,4,17,1.0
2,20,0,0.0,5,4,20,1.0
3,49,0,0.0,9,10,49,1.0
4,30,0,0.0,6,7,30,1.0


Dado que las variables presentan escalas y magnitudes muy diferentes, se va a aplicar una normalización previa al entrenamiento de los modelos. Este proceso evita que las características de mayor rango dominen el aprendizaje y permite que todas las variables contribuyan de forma equilibrada a la detección de accesos atípicos.

In [8]:
#Normalización del dataset para empezar a entrenar a los modelos de aprendizaje automático
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
db_scaled = scaler.fit_transform(db)
db_scaled.shape

(42866142, 7)

## Isolation Forest

Modelo no supervisado que aísla observaciones raras mediante particiones aleatorias.
Es adecuado para detección de outliers en grandes volúmenes y se utiliza cuando no existen etiquetas.
está inspirado en el algoritmo de clasificación y regresión Random Forest.
El modelo Isolation Forest está formado por la combinación de múltiples árboles llamados isolation trees. Estos árboles se crean de forma similar a los de clasificación-regresión: las observaciones de entrenamiento se van separando de forma recursiva creando las ramas del árbol hasta que cada observación queda aislada en un nodo terminal. Sin embargo, en los isolation tree, la selección de los puntos de división se hace de forma aleatoria. Aquellas observaciones con características distintas al resto, quedarán aisladas a las pocas divisiones, por lo que el número de nodos necesarios para llegar a esta observación desde el inicio del árbol (profundidad) es menor que para el resto.

El modelo Isolation Forest se obtiene al combinar múltiples isolation tree, cada uno entrenado con una muestra distinta generada por bootstrapping a partir de los datos de originales. El valor predicho para cada observacion es el número de divisiones promedio que se han necesitado para aislar dicha observacion en el conjunto de árboles. Cuanto menor es este valor, mayor es la probabilidad de que se trate de una anomalía.


In [9]:
from sklearn.ensemble import IsolationForest

# contamination = proporción esperada de anomalías (hiperparámetro)
# Se va a empezar entre el rango de 0.01–0.05
iso = IsolationForest(
    n_estimators=200, #Número de árboles en el bosque
    contamination=0.05, #Proporción esperada de anomalías 
    random_state=42, #Reproducibilidad
    n_jobs=-1 #se usan todos los núcleos de CPu disponibles
)

iso.fit(db_scaled)

t_train_if = time.time() - t0
print("IF entrenado. Tiempo(s):", round(t_train_if, 2))

IF entrenado. Tiempo(s): 415.62


In [10]:
#scoring
t0 = time.time()

# decision_function: mayor => más normal. Menor => más anómalo
df["iso_score"] = iso.decision_function(db_scaled)

# predict: 1 normal, -1 anómalo
df["iso_label"] = iso.predict(db_scaled)

t_score_if = time.time() - t0
print("IF scoring. Tiempo(s):", round(t_score_if, 2))

df[["iso_score", "iso_label"]].describe()
df["iso_label"].value_counts()

IF scoring. Tiempo(s): 481.34


iso_label
 1    40722903
-1     2143239
Name: count, dtype: int64

In [11]:
#Se van a guardar los resultados del entrenamiento para poder evaluarlos posteriormente 
out_if = pd.DataFrame({
    "iso_score": df["iso_score"].values,
    "iso_label": df["iso_label"].values
})

# Se añade trazabilidad si existe (Quién y cuando)
if "dt" in df.columns:
    out_if["dt"] = df["dt"].values
if "src_user" in df.columns:
    out_if["src_user"] = df["src_user"].values

out_if.to_parquet("isolation_forest.parquet", index=False)
print("Isolation Forest guardado:", out_if.shape)

Isolation Forest guardado: (42866142, 4)


In [12]:
import json

run_if_info = {
    "model": "IsolationForest",
    "features": DB_COLS,
    "scaler": "StandardScaler",
    "params": {
        "n_estimators": 200,
        "contamination": 0.05,
        "random_state": 42
    }
}

with open("run_if_info.json", "w") as f:
    json.dump(run_if_info, f, indent=2)

print("Metadatos de IF guardados")

Metadatos de IF guardados


### Hipótesis — Isolation Forest

Se ha entrenado Isolation Forest sobre perfiles horarios y se ha obtenido un score continuo de anomalía, permitiendo generar un ranking Top-K de accesos potencialmente atípicos para revisión.
La distribución del score permite evaluar la presencia de una cola de anomalías, y el Top-K facilita el análisis cualitativo y la estimación de métricas operativas como Precision@K y tasa de falsos positivos mediante revisión manual.

## Autoencoders
Autoencoders es un tipo de red neuronal que se utiliza para el aprendizaje no supervisado. Su función principal es aprender una representación comprimida de los datos de entrada. Consta de dos partes principales: el codificador y el decodificador.

Codificador: Esta parte de la red comprime la entrada en una representación en espacio latente. Codifica los datos de entrada como una representación codificada (comprimida) en una dimensión reducida.
Decodificador: El decodificador tiene como objetivo reconstruir los datos de entrada a partir de la representación codificada. Intenta generar una salida lo más cercana posible a la entrada original.

La idea clave es que autoencoders se entrena para minimizar los errores de reconstrucción, lo que los hace eficientes en el aprendizaje de la distribución de los datos de entrada.

In [13]:
#Preparación de los datos normalizados. Se usa float32 y mini-batches para evitar usar una memoria elevada.
db_ae = db_scaled.astype(np.float32)
db_ae.shape, db_ae.dtype

((42866142, 7), dtype('float32'))

In [14]:
#se importan librerías y se indica donde entrenar el modelo
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [15]:
print("Filas:", db_scaled.shape[0])
print("Features:", db_scaled.shape[1])

# tamaño aproximado de X_ae en RAM si es float32
approx_gb = db_scaled.shape[0] * db_scaled.shape[1] * 4 / (1024**3)
print(f"Tamaño aprox float32: {approx_gb:.2f} GB")


Filas: 42866142
Features: 7
Tamaño aprox float32: 1.12 GB


In [16]:
# Se crea el dataLoader para entregar los datos en lotes (batches)
#Se usa Pytorch para crear este dataset para el modelo
BATCH_SIZE = 4096

dataset = TensorDataset(torch.from_numpy(db_ae))
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, #se mezclan filas para que el modelo no vea patrones por orden temporal
    drop_last=False #para no perder filas
)

Define un autoencoder:

Encoder: comprime el vector de features.
Decoder: intenta reconstruir el vector original.

In [17]:
#Se define el modelo
input_dim = db_ae.shape[1]
latent_dim = max(2, input_dim // 2)

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

ae = AutoEncoder(input_dim, latent_dim).to(device)
ae

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=7, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=3, bias=True)
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=7, bias=True)
  )
)

In [18]:
#Entrenamiento del modelos para minimizar el error de reconstrucción
optimizer = torch.optim.Adam(ae.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

EPOCHS = 5

ae.train()
for epoch in range(1, EPOCHS + 1):
    total_loss = 0.0
    n = 0

    for (batch,) in loader:
        batch = batch.to(device)

        optimizer.zero_grad()
        recon = ae(batch)
        loss = loss_fn(recon, batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.size(0)
        n += batch.size(0)

    print(f"Epoch {epoch}/{EPOCHS} - MSE: {total_loss/n:.6f}")

Epoch 1/5 - MSE: 0.122366
Epoch 2/5 - MSE: 0.015145
Epoch 3/5 - MSE: 0.009545
Epoch 4/5 - MSE: 0.006947
Epoch 5/5 - MSE: 0.006250


In [19]:
#Proceso de scoring
ae.eval()
scores = np.empty(len(db_ae), dtype=np.float32)

with torch.no_grad():
    start = 0
    eval_loader = DataLoader(
        TensorDataset(torch.from_numpy(db_ae)),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

    for (batch,) in eval_loader:
        batch = batch.to(device)
        recon = ae(batch)
        mse = ((recon - batch) ** 2).mean(dim=1).cpu().numpy()

        end = start + len(mse)
        scores[start:end] = mse
        start = end

df["ae_score"] = scores
df["ae_score"].describe()

count    4.286614e+07
mean     5.941479e-03
std      3.710887e-01
min      8.723942e-05
25%      2.492777e-04
50%      4.954672e-04
75%      9.488109e-04
max      5.212789e+02
Name: ae_score, dtype: float64

In [20]:
#Guardar resultado 

out_ae = pd.DataFrame({
    "ae_score": df["ae_score"].values
})

if "dt" in df.columns:
    out_ae["dt"] = df["dt"].values
if "src_user" in df.columns:
    out_ae["src_user"] = df["src_user"].values

out_ae.to_parquet("autoencoders.parquet", index=False)
print("Autoencoder guardado:", out_ae.shape)

Autoencoder guardado: (42866142, 3)


In [21]:
#Guardar metadatos del experimento
import json

run_ae_info = {
    "model": "Autoencoder",
    "features": DB_COLS,
    "scaler": "StandardScaler",
    "params": {
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "latent_dim": latent_dim,
        "optimizer": "Adam",
        "loss": "MSE"
    }
}

with open("run_ae_info.json", "w") as f:
    json.dump(run_ae_info, f, indent=2)

print("Metadatos del Autoencoder guardados")


Metadatos del Autoencoder guardados


## LOF
Como este modelo se utiliza para una base de datos con un menor volumen, vamos a entrenarlos por separado con un muestreo sacado de la propia base de datos que se preparo en una etapa anterior.

In [27]:
#muestreo aleatorio reproducible.

RANDOM_STATE = 42
SAMPLE_N = 1_000_000

rng = np.random.RandomState(RANDOM_STATE)
sample_idx = rng.choice(db_scaled.shape[0], size=SAMPLE_N, replace=False)

db_muestra = db_scaled[sample_idx].astype(np.float32, copy=False)
print("Muestra:", db_muestra.shape, db_muestra.dtype)

Muestra: (1000000, 7) float32


In [28]:
sample_meta = pd.DataFrame({"row_id": sample_idx})

# trazabilidad opcional si existe en df
for c in ["dt", "src_user"]:
    if c in df.columns:
        sample_meta[c] = df.loc[sample_idx, c].values

sample_meta.to_parquet("sample_idx.parquet", index=False)
print("Guardado: sample_idx.parquet", sample_meta.shape)

Guardado: sample_idx.parquet (1000000, 3)


In [29]:
#LOF “normal” (novelty=False) está pensado para fit_predict en el mismo set, pero para tener un score usable y consistente, mejor novelty=True.
import time

from sklearn.neighbors import LocalOutlierFactor

LOF_NEIGHBORS = 35
LOF_CONTAM = 0.02

t0 = time.time()

lof = LocalOutlierFactor(
    n_neighbors=LOF_NEIGHBORS,
    contamination=LOF_CONTAM,
    novelty=True,     
    n_jobs=-1
)
lof.fit(db_muestra)

t_train_lof = time.time() - t0
print("LOF entrenado. Tiempo(s):", round(t_train_lof, 2))

LOF entrenado. Tiempo(s): 213.46


In [30]:
#scoring
t0 = time.time()

lof_score_raw = lof.decision_function(db_muestra)  # mayor => más normal
lof_score = -lof_score_raw                  # mayor => más anómalo
lof_label = lof.predict(db_muestra)                # 1 normal, -1 anómalo

t_score_lof = time.time() - t0
print("LOF scoring. Tiempo(s):", round(t_score_lof, 2))

pd.Series(lof_label).value_counts()

LOF scoring. Tiempo(s): 423.5


 1    981525
-1     18475
Name: count, dtype: int64

In [31]:
#guardar datos
out_lof = sample_meta.copy()
out_lof["lof_score"] = lof_score.astype(np.float32)
out_lof["lof_label"] = lof_label.astype(np.int8)

out_lof.to_parquet("lof.parquet", index=False)
print("Guardado: lof.parquet", out_lof.shape)

Guardado: lof.parquet (1000000, 5)


In [32]:
#Guardar metadatos
import json
run_sample_info = {
    "sample": {"n": int(SAMPLE_N), "random_state": int(RANDOM_STATE)},
    "features": DB_COLS,
    "scaler": "StandardScaler",
    "lof": {
        "n_neighbors": int(LOF_NEIGHBORS),
        "contamination": float(LOF_CONTAM),
        "novelty": True,
        "train_seconds": float(t_train_lof),
        "score_seconds": float(t_score_lof)
    }
}

with open("run_lof_info.json", "w") as f:
    json.dump(run_sample_info, f, indent=2)

print("Guardado: run_lof_info.json")

Guardado: run_lof_info.json
